# ⛔️ Traffic Signs Classification with Resnet50

<h2>What is Transfer Learning</h2>
<center><img width=600px src = "https://ruder.io/content/images/2017/03/transfer_learning_setup.png" alt="Transfer Learning"></center>

<p>Transfer learning allows us to deal with these scenarios by leveraging the already existing labeled data of some related task or domain. We try to store this knowledge gained in solving the source task in the source domain and apply it to our problem of interest as can be seen in Figure above </p>

<div id="resnet"><h3>What is A ResNet 50?</h3></div>
<p></p>
<center><img src="https://i.stack.imgur.com/gI4zT.png" width=600px alt="ResNet"></center>
<p></p>
<p>
ResNet, short for Residual Networks is a classic neural network used as a backbone for many computer vision tasks. This model was the winner of ImageNet challenge in 2015. The fundamental breakthrough with ResNet was it allowed us to train extremely deep neural networks with 150+layers successfully. Prior to ResNet training very deep neural networks was difficult due to the problem of vanishing gradients.</p>

# 📥 Importing needed libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
from timeit import default_timer as timer

import tensorflow as tf

from keras.utils.np_utils import to_categorical
from keras.models import Sequential # A Sequential model is appropriate for a plain 
#stack of layers where each layer has exactly one input tensor and one output tensor.

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape, Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.applications import ResNet50


import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(os.listdir('../input'))

In [2]:
# Fixed for our Categories classes
NUM_CLASSES = 43

# Fixed for Categories color images
CHANNELS = 3

IMAGE_RESIZE = 224
#first resize all the images to 224 shape and then learn

RESNET50_POOLING_AVERAGE = 'avg'

DENSE_LAYER_ACTIVATION = 'softmax'
#Softmax converts a vector of values to a probability distribution.The elements of the output vector are in range (0, 1) and sum to 1.

OBJECTIVE_FUNCTION = 'categorical_crossentropy'
#categorical_crossentropy: Used as a loss function for multi-class classification model 
#where there are two or more output labels. The output label is assigned one-hot category encoding value in form of 0s and 1.

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# Too many epochs can lead to overfitting of the training dataset, whereas too few may result in an underfit model. 
# Early stopping is a method that allows you to specify an arbitrary large number of training epochs and stop training 
# once the model performance stops improving on a hold out validation dataset.
# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

# 📂 Loading dataset data2.pickle with RGB examples

In [3]:
# Opening file for reading in binary mode
with open('../input/traffic-signs-preprocessed/data2.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type

# Preparing y_train and y_validation for using in Keras
# num_classes: Total number of classes. If nothing is mentioned, it considers 
# the largest number of the input vector and adds 1, to get the number of classes.Its default value is "None".

# to_categorical represent different categories
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
# transpose function is axes of data
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)


# 💫 Showing some examples

In [12]:
%matplotlib inline

# Preparing function for ploting set of examples
# As input it will take 4D tensor and convert it to the grid
# Values will be scaled to the range [0, 255]

def convert_to_grid(x_input):
    N, H, W, C = x_input.shape
    # ceil() is a mathematical function that returns the ceil of the elements of array. The ceil of the scalar x is the smallest integer i, such that i >= x
    grid_size = int(np.ceil(np.sqrt(N)))
    grid_height = H * grid_size + 1 * (grid_size - 1)
    grid_width = W * grid_size + 1 * (grid_size - 1)
    # Return a new array of given shape and type, filled with zeros.
    grid = np.zeros((grid_height, grid_width, C)) + 255
    next_idx = 0
    y0, y1 = 0, H
    
    for y in range(grid_size):
        x0, x1 = 0, W
        
        for x in range(grid_size):
            if next_idx < N:
                img = x_input[next_idx]
                low, high = np.min(img), np.max(img)
                grid[y0:y1, x0:x1] = 255.0 * (img - low) / (high - low)
                next_idx += 1
            x0 += W + 1
            x1 += W + 1
        y0 += H + 1
        y1 += H + 1

    return grid


# Visualizing some examples of training data
examples = data['x_train'][:81, :, :, :]
print(examples.shape)  # (81, 32, 32, 3)

# Plotting some examples
fig = plt.figure()
grid = convert_to_grid(examples)
# A UINT8 is an 8-bit unsigned integer
plt.imshow(grid.astype('uint8'), cmap='gray')
plt.axis('off')
# GCF means Get the current figure.
plt.gcf().set_size_inches(15, 15)
plt.title('Some examples of training data', fontsize=18)

# Showing the plot
plt.show()

# Saving the plot
fig.savefig('training_examples.png')
plt.close()


# 🏗️ Building model of Resnet50 (Transfer Learning)

In [5]:
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [6]:
img_size = (224,224)
model = Sequential()
# “include_top” argument can be set to False, in which case the fully-connected output layers of the model used to make predictions is not loaded, allowing a new output layer to be added and trained. 

model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))
model.add(Dropout(0.1))
# The rectified linear activation function or ReLU for short is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero.
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
model.layers[2].trainable = False

# 🤏 Training Model

In [7]:
from keras import optimizers

# Stochastic gradient descent optimizer.

# Includes support for momentum, learning rate decay, and Nesterov momentum.

# Arguments

# lr: float >= 0. Learning rate.
# momentum: float >= 0. Parameter updates momentum.
# decay: float >= 0. Learning rate decay over each update.
# nesterov: boolean. Whether to apply Nesterov momentum.
    
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [8]:
hist = model.fit(data['x_train'], data['y_train'], validation_data =(data['x_validation'], data['y_validation']), epochs = 20, batch_size = 1000)

In [9]:
print('Epochs={0:d}, training accuracy={1:.5f}, validation accuracy={2:.5f}'.\
      format(10, max(hist.history['acc']), max(hist.history['val_acc'])))

# 📈 Plotting results

In [10]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 5.0) # Setting default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['font.family'] = 'Times New Roman'

fig = plt.figure()
plt.plot(hist.history['acc'], '-o', linewidth=3.0)
plt.plot(hist.history['val_acc'], '-o', linewidth=3.0)
plt.title('Overfitting small data', fontsize=22)
plt.legend(['train', 'validation'], loc='upper left', fontsize='xx-large')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)
plt.tick_params(labelsize=18)

# Showing the plot
plt.show()

# Saving the plot
fig.savefig('overfitting_small_data.png')
plt.close()


# 🖼️ Predicting with one image from test dataset

In [11]:
%matplotlib inline

# Preparing image for predicting from test dataset
x_input = data['x_test'][100:101]
print(x_input.shape)
y_input = data['y_test'][100:101]
print(y_input)

plt.rcParams['figure.figsize'] = (2.5, 2.5) # Setting default size of plots
plt.imshow(x_input[0, :, :, :])
plt.axis('off')

# Showing the plot
plt.show()

# Getting scores from forward pass of input image
scores = model.predict(x_input)
print(scores.shape) # (43,)

# Scores is given for image with 43 numbers of predictions for each class
# Getting only one class with maximum value
prediction = np.argmax(scores)
print('ClassId:', prediction)

# Defining function for getting texts for every class - labels
def label_text(file):
    # Defining list for saving label in order from 0 to 42
    label_list = []
    
    # Reading 'csv' file and getting image's labels
    r = pd.read_csv(file)
    # Going through all names
    for name in r['SignName']:
        # Adding from every row second column with name of the label
        label_list.append(name)
    
    # Returning resulted list with labels
    return label_list


# Getting labels
labels = label_text('../input/traffic-signs-preprocessed/label_names.csv')

# Printing label for classified Traffic Sign
print('Label:', labels[prediction])
